In [1]:
import sqlite3
import pandas as pd
import io

In [2]:
with open("database.sql", "r") as file:
    sql_script = file.read()

connection = sqlite3.connect(":memory:")

with connection:
    connection.executescript(sql_script)

In [3]:
query = """
            SELECT * 
            FROM companhia_aerea
        """

df = pd.read_sql_query(query, connection)
df

,id_companhia_aerea,nome_companhia,regra_tarifaria
0,1,LATAM AIRLINES BRASIL,None
1,2,GOL Linhas Aéreas Inteligentes,https://www.voegol.com.br/pt/informacoes/tarifas
2,3,AZUL LINHAS AÉREAS BRASILEIRA,https://www.voeazul.com.br/para-sua-viagem/inf...


## Consulta 1: Órgão que comprou o bilhete mais caro gerado pela LATAM

In [4]:
query = """
            SELECT DISTINCT codigo_orgao, nome_orgao, valor_bilhete
            FROM orgao
            NATURAL JOIN orgao_bilhete
            NATURAL JOIN bilhete
            NATURAL JOIN companhia_aerea
            WHERE nome_companhia == "LATAM AIRLINES BRASIL"
            ORDER BY
                valor_bilhete DESC
            LIMIT 1
            
        """

df = pd.read_sql_query(query, connection)
df

,codigo_orgao,nome_orgao,valor_bilhete
0,78055,Secretaria Especial de Articulação e Monitoram...,5847.14


## Consulta 2: Aviões que tem mais de 200 assentos

In [5]:
query = """
            SELECT DISTINCT id_aviao, modelo
            FROM aviao
            WHERE assentos > 200
        """

df = pd.read_sql_query(query, connection)
df

,id_aviao,modelo
0,4,Airbus A321
1,5,Airbus A321neo
2,6,Airbus A330-200
3,7,Airbus A330-900
4,15,Boeing 777-300ER
5,16,Boeing 787-9


## Consulta 3: Companhias aereas que não possuem nenhum avião com o selo B da anac


In [6]:
query = """
            SELECT DISTINCT id_companhia_aerea
            FROM companhia_aerea_aviao
            WHERE id_companhia_aerea NOT IN (
                SELECT id_companhia_aerea
                FROM companhia_aerea_aviao
                NATURAL JOIN aviao
                WHERE selo_anac = "B")
            
        """

df = pd.read_sql_query(query, connection)
df

,id_companhia_aerea
0,2


## Consulta 4: Orgão cuja soma das multas com bilhetes aéreos é maior

In [7]:
query = """
            SELECT DISTINCT codigo_orgao, nome_orgao, SUM(valor_multas) as soma_das_multas
            FROM orgao
            NATURAL JOIN orgao_bilhete
            NATURAL JOIN bilhete
            WHERE valor_multas > 0 
            GROUP BY codigo_orgao, nome_orgao
            ORDER BY soma_das_multas desc
            LIMIT 1
            
        """

df = pd.read_sql_query(query, connection)
df

,codigo_orgao,nome_orgao,soma_das_multas
0,70352,Secretaria-Geral de Administração,9562.89


## Consulta 5: Órgão que não comprou bilhetes aéreos

In [8]:
query = """
            SELECT codigo_orgao, nome_orgao
            FROM orgao
            WHERE codigo_orgao NOT IN (
                SELECT codigo_orgao
                FROM orgao
                NATURAL JOIN orgao_bilhete
            )
            
        """

df = pd.read_sql_query(query, connection)
df

,codigo_orgao,nome_orgao
0,26,Presidência da República
1,27,Gabinete de Segurança Institucional da Presidê...
2,46,Advocacia-Geral da União
3,2837,Casa Civil da Presidência da República
4,42673,Secretaria de Relações Institucionais
5,264359,Ministério das Comunicações
6,308798,Ministério das Cidades
7,308799,Ministério da Integração e do Desenvolvimento ...
8,308800,Ministério da Fazenda
9,308803,Ministério da Gestão e da Inovação em Serviços...


## Consulta 6: Número de viagens de cada órgão superior

In [9]:
query = """
            SELECT nome_superior, SUM(nViagens) as tViagens
            FROM (
                SELECT o2.nome_orgao as nome_superior, COUNT(DISTINCT b.localizador) AS nViagens
                FROM orgao AS o1
                JOIN orgao AS o2 ON o1.codigo_orgao_superior = o2.codigo_orgao
                JOIN orgao_bilhete AS ob ON o1.codigo_orgao = ob.codigo_orgao
                JOIN bilhete AS b ON ob.localizador = b.localizador
                WHERE o2.codigo_orgao_superior IS NULL AND b.no_show = FALSE AND b.cancelado = FALSE
                GROUP BY o1.codigo_orgao
            ) AS subquery
            GROUP BY nome_superior
            ORDER BY tViagens DESC
        """

df = pd.read_sql_query(query, connection)
df

,nome_superior,tViagens
0,Ministério dos Direitos Humanos e da Cidadania,1264
1,Gabinete de Segurança Institucional da Presidê...,380
2,Secretaria de Relações Institucionais,258
3,Ministério das Cidades,238
4,Ministério da Fazenda,218
5,Advocacia-Geral da União,207
6,Ministério da Integração e do Desenvolvimento ...,197
7,Casa Civil da Presidência da República,173
8,Secretaria-Geral da Presidência da República,150
9,Ministério da Gestão e da Inovação em Serviços...,141


## Consulta 7: Bilhete mais caro

In [11]:
query = """
            SELECT localizador, valor_bilhete, data_embarque
            FROM bilhete
            WHERE no_show = FALSE OR cancelado = FALSE
            ORDER BY valor_bilhete DESC
            LIMIT 1
        """

df = pd.read_sql_query(query, connection)
df

,localizador,valor_bilhete,data_embarque
0,FTWGKW,5847.14,30/06/2024


## Consulta 8: Aviões ativos

In [12]:
query = """
            SELECT nome_companhia, modelo, assentos, ativos
            FROM companhia_aerea
            NATURAL JOIN companhia_aerea_aviao 
            NATURAL JOIN aviao
            WHERE ativos > 0
        """

df = pd.read_sql_query(query, connection)
df

,nome_companhia,modelo,assentos,ativos
0,LATAM AIRLINES BRASIL,Airbus A319,144,19
1,LATAM AIRLINES BRASIL,Airbus A320,174,60
2,LATAM AIRLINES BRASIL,Airbus A320neo,174,16
3,LATAM AIRLINES BRASIL,Airbus A321,224,31
4,LATAM AIRLINES BRASIL,Airbus A321neo,228,14
5,LATAM AIRLINES BRASIL,Boeing 777-300ER,410,10
6,LATAM AIRLINES BRASIL,Boeing 787-9,303,1
7,GOL Linhas Aéreas Inteligentes,Boeing 737-700,138,11
8,GOL Linhas Aéreas Inteligentes,Boeing 737-800,186,56
9,GOL Linhas Aéreas Inteligentes,Boeing 737 MAX 8,186,52


## Consulta 9: Companhias aéreas com mais cancelamentos e/ou no shows

In [13]:
query = """
            SELECT nome_companhia, COUNT(DISTINCT localizador) as nBilhetes, 
            COUNT(DISTINCT CASE WHEN no_show = TRUE OR cancelado = TRUE THEN localizador END) as nCancelamentos,
            ROUND(
                100.0 * COUNT(DISTINCT CASE WHEN no_show = TRUE OR cancelado = TRUE THEN localizador END) / COUNT(DISTINCT localizador), 2) AS pctCancelamentos
            FROM companhia_aerea
            NATURAL JOIN bilhete
            GROUP BY id_companhia_aerea
            ORDER BY pctCancelamentos DESC
        """

df = pd.read_sql_query(query, connection)
df

,nome_companhia,nBilhetes,nCancelamentos,pctCancelamentos
0,AZUL LINHAS AÉREAS BRASILEIRA,771,80,10.38
1,LATAM AIRLINES BRASIL,1938,143,7.38
2,GOL Linhas Aéreas Inteligentes,1245,69,5.54


## Consulta 10: Órgãos que mais cancelaram ou não apareceram em bilhetes comprados

In [14]:
query = """
            SELECT nome_superior, SUM(nViagens) as cancelamentos
            FROM (
                SELECT o2.nome_orgao as nome_superior, COUNT(DISTINCT b.localizador) AS nViagens
                FROM orgao AS o1
                JOIN orgao AS o2 ON o1.codigo_orgao_superior = o2.codigo_orgao
                JOIN orgao_bilhete AS ob ON o1.codigo_orgao = ob.codigo_orgao
                JOIN bilhete AS b ON ob.localizador = b.localizador
                WHERE o2.codigo_orgao_superior IS NULL AND (b.no_show = TRUE OR b.cancelado = TRUE)
                GROUP BY o1.codigo_orgao
            ) AS subquery
            GROUP BY nome_superior
            ORDER BY cancelamentos DESC
        """

df = pd.read_sql_query(query, connection)
df

,nome_superior,cancelamentos
0,Advocacia-Geral da União,48
1,Gabinete de Segurança Institucional da Presidê...,38
2,Ministério dos Direitos Humanos e da Cidadania,37
3,Secretaria de Relações Institucionais,30
4,Ministério da Gestão e da Inovação em Serviços...,24
5,Ministério das Cidades,19
6,Secretaria-Geral da Presidência da República,18
7,Ministério da Integração e do Desenvolvimento ...,16
8,Ministério da Fazenda,14
9,Casa Civil da Presidência da República,14
